In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

In [2]:
url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
krx = pd.read_html(url, header=0)[0]
krx = krx[['종목코드', '회사명']]
krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
krx.code = krx.code.map('{:06d}'.format)
krx.head()

,code,company
0,095570,AJ네트웍스
1,282330,BGF리테일
2,138930,BNK금융지주
3,155660,DSR
4,294870,HDC현대산업개발


In [16]:
pages_to_fetch = 2
company = '삼성전자'
code = '005930'
headers= {'USER-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
response = requests.get(url, headers=headers)
html = BeautifulSoup(response.content, 'lxml')
pgrr = html.find("td", class_="pgRR")
s = str(pgrr.a["href"]).split('=')
lastpage = s[-1]
df = pd.DataFrame()
pages = min(int(lastpage), pages_to_fetch)
for page in range(1, pages + 1):
    pg_url = '{}&page={}'.format(url, page)
    pg_response = requests.get(pg_url, headers=headers)
    df_new = pd.read_html(pg_response.content, encoding='cp949', header=0)[0]
    df = pd.concat([df, df_new])
    tmnow = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
            format(tmnow, company, code, page, pages), end="\r")
df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
df['date'] = df['date'].replace('.', '-')
df = df.dropna()
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
df.head()


,date,open,high,low,close,diff,volume
1,2023.11.17,72300,73000,72300,72500,300,10845642
2,2023.11.16,72500,73000,72300,72800,600,15860451
3,2023.11.15,71600,72200,71500,72200,1400,20148677
4,2023.11.14,71000,71100,70600,70800,400,9567984
5,2023.11.13,71300,71300,70300,70400,100,9246919
